In [1]:
# required libraries
!pip install datasets

import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


# Model loading and tokenizer / Data Loading and Preprocessing

In [2]:
sia = SentimentIntensityAnalyzer()

random.seed(42)
torch.manual_seed(42)

def load_model_and_tokenizer_sentiment():
    """Initialize the model and tokenizer with special tokens."""
    tokenizer = AutoTokenizer.from_pretrained('gpt2')
    model = AutoModelForCausalLM.from_pretrained('gpt2')

    # Add special tokens, '<sent>', '<sent_pos>', '<sent_neg>', '<text>' for VADER
    special_tokens = {'additional_special_tokens': ['<sent>', '<sent_pos>', '<sent_neg>', '<text>']}
    num_added_tokens = tokenizer.add_special_tokens(special_tokens)

    # Important: resize model embeddings to account for new tokens
    model.resize_token_embeddings(len(tokenizer))

    # Set padding token
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# Initialize the model and tokenizer
model_sent, tokenizer_sent = load_model_and_tokenizer_sentiment()


def load_training_data(percentage): # changed to 0.3%, 33000 examples -> 99500 examples
    """Load a subset of OpenWebText dataset.
    Start with a small percentage (0.1%) for testing."""

    dataset = load_dataset("Bingsu/openwebtext_20p", split="train") # OpenWebText dataset
    num_samples = len(dataset)
    indices = random.sample(range(num_samples), int(percentage * num_samples))
    subset = dataset.select(indices)
    return subset['text']

# Example usage:
texts = load_training_data(0.003)
print(f"Loaded {len(texts)} training examples.")

def process_sentence(sentence):
    sentence = sentence.strip().replace("\n", " ")
    sentiment_score = sia.polarity_scores(sentence)['compound']
    if sentiment_score >= 0:
        sentiment_label = "<sent_pos>"
    else:
        sentiment_label = "<sent_neg>"
    return f"<sent> {sentiment_label} <text> {sentence}"

# process all sentences
processed_texts_sent = [process_sentence(text) for text in texts]
print("Sample processed sentiment text:")
print(processed_texts_sent [0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

(…)-00015-of-00017-d88f6095d9439e35.parquet:   0%|          | 0.00/290M [00:00<?, ?B/s]

(…)-00016-of-00017-55bc94e70d1f8efa.parquet:   0%|          | 0.00/290M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/33167823 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

Loaded 99503 training examples.
Sample processed sentiment text:
<sent> <sent_pos> <text> Challenging the state forces, Anu said they were able to go to the people and talk to them, even in places where government has deployed 300 to 400 security personnel. "This definitely shows people's support for us,” she said, exuding confidence that they will succeed in broadening their base among the people," Anu said.


# Setup Training with HuggingFace Trainer / Fine-tuning the Model

In [ ]:
class SentimentControlledDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # tokenize the processed text
        encoding = self.tokenizer(self.texts[idx], truncation=True, max_length=self.max_length, padding='max_length', return_tensors='pt')
        # squeeze to remove extra batch dimension
        item = {key: tensor.squeeze() for key, tensor in encoding.items()}
        return item

# create the sentiment dataset and DataLoader
train_dataset_sent = SentimentControlledDataset(processed_texts_sent, tokenizer_sent)
print(f"Sentiment dataset size: {len(train_dataset_sent)} examples.")

# data collator which handles language modeling
data_collator_sent = DataCollatorForLanguageModeling(tokenizer=tokenizer_sent, mlm=False)

training_args_sent = TrainingArguments(
    output_dir="./gpt2_sentiment_control",
    overwrite_output_dir=True,
    num_train_epochs=5, # epochs (5 for 100,000 samples)
    per_device_train_batch_size=8, # small batch size (4 to 8)
    gradient_accumulation_steps=2,
    save_steps=5000, # frequency of saving checkpoints
    save_total_limit=2, # keep only the latest 2 checkpoints
    logging_steps=100, # log training loss every 100 steps
    learning_rate=3e-5, # learning rate for stable training
    warmup_steps=500, # warmup steps for learning rate scheduler
    report_to="tensorboard" # "tensorboard" for visualization
)

# initialize trainer
trainer_sent = Trainer(
    model=model_sent,
    args=training_args_sent,
    data_collator=data_collator_sent,
    train_dataset=train_dataset_sent
)

# start training
trainer_sent.train()

# save the final model
trainer_sent.save_model("./gpt2_sentiment_control_final")

Sentiment dataset size: 99503 examples.


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,4.476800
200,4.059100
300,3.920800
400,3.777200
500,3.561200
600,3.487200
700,3.486300
800,3.476400
900,3.424000
1000,3.402800


# Sentiment-Controlled Generation

In [10]:
import torch
from tqdm import tqdm
from transformers import StoppingCriteria, StoppingCriteriaList

#  sentiment of the generated texts using VADER
def evaluate_sentiment(generated_text):
    score = sia.polarity_scores(generated_text)['compound']
    if score >= 0:
        label = "positive"
    else:
        label = "negative"
    return label, score


def generate_sentiment_text(prompt, tokenizer, model, desired_sentiment, max_length=128, temperature=0.3, num_retries=3):
    for attempt in range(num_retries):
        encoding = tokenizer(prompt, return_tensors='pt', padding=True)
        input_ids = encoding.input_ids.to(model.device)
        attention_mask = encoding.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_return_sequences=1,
                do_sample=True,
                temperature=temperature, # temperature for randomness
                pad_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.2, # repetition penalty
                top_k=50, # limit to top-50 tokens
                top_p=0.95 # nucleus sampling to keep cumulative probability at 95%
            )

        generated = tokenizer.decode(outputs[0], skip_special_tokens=False)
        # remove the prompt from the output if present
        if generated.startswith(prompt):
            generated = generated[len(prompt):].strip()

        # evaluate the sentiment of the generated text
        label, score = evaluate_sentiment(generated)
        if (desired_sentiment == "positive" and score > 0) or (desired_sentiment == "negative" and score < 0):
            break
    return generated

prompt_positive = "<sent> <sent_pos> <text>"
prompt_negative = "<sent> <sent_neg> <text>"

generated_positive = generate_sentiment_text(prompt_positive, tokenizer_sent, model_sent, desired_sentiment="positive", max_length=128, temperature=0.3, num_retries=3)
generated_negative = generate_sentiment_text(prompt_negative, tokenizer_sent, model_sent, desired_sentiment="negative", max_length=128, temperature=0.3, num_retries=3)

print("Positive sentiment generation:")
print("Prompt:", prompt_positive)
print("Generated:", generated_positive)

print("\nNegative sentiment generation:")
print("Prompt:", prompt_negative)
print("Generated:", generated_negative)

label_pos, score_pos = evaluate_sentiment(generated_positive)
label_neg, score_neg = evaluate_sentiment(generated_negative)

print("\nEvaluation of Generated Texts:")
print(f"Positive prompt generated text classified as: {label_pos} (compound score: {score_pos})")
print(f"Negative prompt generated text classified as: {label_neg} (compound score: {score_neg})")


Positive sentiment generation:
Prompt: <sent> <sent_pos> <text>
Generated: The second part of the story is a very interesting and interesting one. It’s about how the government has been able to use its surveillance powers to spy on people in the United States, as well it also describes how the government uses these surveillance powers to monitor Americans abroad: “The government can use this information to target anyone who travels to America, anywhere in the world that they want; any person whose passport or other documents are being searched; anyone who visits their home country for any reason; anyone who goes to an FBI office in the U.[9] This is a great example because it shows how

Negative sentiment generation:
Prompt: <sent> <sent_neg> <text>
Generated: The U.S.-led coalition of Arab and Muslim countries has been bombing ISIS targets in Syria for months, but the group’s fighters are still fighting alongside al Qaeda and other terrorist groups to take territory from their strongh